# **INF161 - Bike Traffic Prediction Modelling**
### *Ole Kristian Westby | owe009@uib.no | H23*

In this notebook, I will go through modelling and prediction for this project.


In [196]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dir = 'ready_data/ready_data.csv'

# Load data into a dataframe
df = pd.read_csv(dir)

# Look at first rows
df.head()

,Datotid,Trafikkmengde,Globalstraling,Solskinstid,Lufttemperatur,Vindstyrke,Lufttrykk,Vindkast
0,2015-07-16 15:00:00,50,504.400000,7.233333,13.900000,4.083333,1014.433333,6.70
1,2015-07-16 16:00:00,101,432.833333,8.116667,13.733333,4.333333,1014.400000,7.20
2,2015-07-16 17:00:00,79,378.400000,10.000000,13.866667,3.933333,1014.066667,6.55
3,2015-07-16 18:00:00,56,212.583333,10.000000,13.216667,4.233333,1013.966667,7.15
4,2015-07-16 19:00:00,45,79.750000,10.000000,12.683333,2.950000,1014.100000,5.45


Let's check for missing data.

In [197]:
# Missing data
missing_data = df.isnull().sum()
missing_data

Datotid             0
Trafikkmengde       0
Globalstraling    403
Solskinstid       403
Lufttemperatur    403
Vindstyrke        403
Lufttrykk         403
Vindkast          403
dtype: int64

Now, regarding the missing values in the dataframe, we're gonna fix this by imputing them with the median because it's less sensitive to outliers. This will artifically impute these missing values with the median of the other values so it's sort of accurate. There are other strategies to do this, and they're not necessarily false either, but I choose the median because of its insensitivity to outliers.

In [198]:
# Impute with median
for col in missing_data.index[missing_data > 0]:
    df[col].fillna(df[col].median(), inplace=True)

# Now we can check again
missing_data = (df.isnull().sum())
missing_data

Datotid           0
Trafikkmengde     0
Globalstraling    0
Solskinstid       0
Lufttemperatur    0
Vindstyrke        0
Lufttrykk         0
Vindkast          0
dtype: int64

Next, we'll convert the "Datotid" column to datetime format, and extract month, day and hour from the dataframe. We will also be doing the feature engineering part to increase our model's accuracy.

In [199]:
import holidays # Holidays feature engineering

df['Datotid'] = pd.to_datetime(df['Datotid'])

# Extracting time-related stuff
df['Month'] = df['Datotid'].dt.month
df['DayOfWeek'] = df['Datotid'].dt.dayofweek # Monday: 0, Tuesday: 1, ... , Sunday: 6.
df['Hour'] = df['Datotid'].dt.hour

# Feature engineering: Public holidays in Norway
norway_holidays = holidays.Norway()
df['IsHoliday'] = df['Datotid'].apply(lambda x: pd.to_datetime(x).date() in norway_holidays)

# Feature engineering: Weekends, rushhour. I tested different options on "rushhour" and found this to be the best
df['IsWeekend'] = df['Datotid'].dt.dayofweek >= 5
df['IsRushhour'] = df['Hour'].isin([7, 8, 15, 16, 17])

df['IsNight'] = df['Hour'].isin([0, 1, 2, 3, 4, 5])

# Feature engineering: Seasons
df['Summer'] = df['Month'].isin([6, 7, 8])
df['Winter'] = df['Month'].isin([12, 1, 2])
df['Spring'] = df['Month'].isin([3, 4, 5])
df['Autumn'] = df['Month'].isin([9, 10, 11])

df.reset_index(drop=True, inplace=True)

print(df)

                  Datotid  Trafikkmengde  Globalstraling  Solskinstid  \
0     2015-07-16 15:00:00             50      504.400000     7.233333   
1     2015-07-16 16:00:00            101      432.833333     8.116667   
2     2015-07-16 17:00:00             79      378.400000    10.000000   
3     2015-07-16 18:00:00             56      212.583333    10.000000   
4     2015-07-16 19:00:00             45       79.750000    10.000000   
...                   ...            ...             ...          ...   
65356 2022-12-31 19:00:00              0       -0.400000     0.000000   
65357 2022-12-31 20:00:00              0       -0.150000     0.000000   
65358 2022-12-31 21:00:00              3       -1.750000     0.000000   
65359 2022-12-31 22:00:00              5       -0.933333     0.000000   
65360 2022-12-31 23:00:00              1       -3.983333     0.000000   

       Lufttemperatur  Vindstyrke    Lufttrykk  Vindkast  Month  DayOfWeek  \
0           13.900000    4.083333  1014.43333

Now that our data is clean and *really ready*, we can start with data splitting. We will first need to import an additional library for this.

In [200]:
from sklearn.model_selection import train_test_split

# X: all weather information, y: only trafikkmengde.
X = df.drop(columns=['Datotid', 'Trafikkmengde'])
y = df['Trafikkmengde']

# We want equal training and validation sets (samples)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.5, random_state=1)

# How many samples?
X_train.shape, X_val.shape

((32680, 18), (32681, 18))

The data has now been sucessfully split into a training set, and a validation set. The training set contains 32,680 samples and the validation set contains 32,681 samples.

Next, we need to select a model, and we'll need to import numpy, as well as the first model and MSE. Let's try Linear Regression as our first model. ~0s runtime

In [201]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Initialize the model and train it
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict on the validation set
lr_predict = lr.predict(X_val)

# Calulate RMSE (root of mean squared error)
lr_rmse = np.sqrt(mean_squared_error(y_val, lr_predict))

lr_rmse

46.26945543379749

The RMSE for this model is ~46.3 approx. Before we continue checking different models, I wish to stop here and think a bit. Is ~43,3 a good RMSE here? I think in order to evaluate this, we need to check how many cycle on average each day. Let's check this.

In [202]:
# Looking at all trafikkmengde stats.
cycle_stats = df['Trafikkmengde'].describe()
cycle_stats

count    65361.000000
mean        50.379905
std         69.782243
min          0.000000
25%          5.000000
50%         25.000000
75%         64.000000
max        608.000000
Name: Trafikkmengde, dtype: float64

Given this, it is clear that an RMSE of ~46,3 is pretty high, especially since the 75th percentile is 64 cycles. On average we're gonna be really off with this. This model won't do, we need to explore some more.. We can try the Lasso Regression model from the lab works. ~0.1s runtime

In [203]:
from sklearn.linear_model import Lasso

# Initialize the model and train it
lasso = Lasso(alpha=0.1, random_state=1)
lasso.fit(X_train, y_train)

# Predict on val set
lasso_predict = lasso.predict(X_val)

# RMSE
lasso_rmse = np.sqrt(mean_squared_error(y_val, lasso_predict))
lasso_rmse


46.28320042020932

Hmm.. that's even worse (not that much more worse!). So far the Linear Regression model is winning, but there has to be a better one.. How about Random Forest? ~5.5s runtime

In [204]:
from sklearn.ensemble import RandomForestRegressor

# Initialize the model and train it
rf = RandomForestRegressor(n_estimators=10, random_state=1) # With 100 estimators, the RMSE is 26.1455 instead, but it took a minute to run. # 1000: 25,89

rf.fit(X_train, y_train)

# Predict on val set
rf_predict = rf.predict(X_val)

# RMSE
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_predict))
rf_rmse

23.80429214106913

Woah! That's a big improvement. Definitely the best model so far with RMSE ~23.8. Will try GradientBoosting, ~0.7s runtime

In [205]:
from sklearn.ensemble import GradientBoostingRegressor

# Initialize the model and train it
gb = GradientBoostingRegressor(n_estimators=10, random_state=1)
gb.fit(X_train, y_train)

# Predict on val set
gb_predict = gb.predict(X_val)

# RMSE
gb_rmse = np.sqrt(mean_squared_error(y_val, gb_predict))
gb_rmse

43.78259500701312

Hmm, not better. Okay, out of the three models we tested so far, RandomForest is winning. I'll try some more.. ps: in the final deadline I will probably make adjustments so it will check all models at once for the best one instead of individually like this, this is just for visualization and thought-descriptions. Next will try ElasticNet, ~0s runtime

In [206]:
from sklearn.linear_model import ElasticNet

# Initialize the model and train it
elastic_net = ElasticNet(alpha=1, random_state=1)
elastic_net.fit(X_train, y_train)

# Predict on val set
y_predict = elastic_net.predict(X_val)

# RMSE
en_rmse = np.sqrt(mean_squared_error(y_val, y_predict))
en_rmse

56.05408475036874

Let's try a different type of model, KNeighborsRegressor. ~1 second runtime

In [207]:
from sklearn.neighbors import KNeighborsRegressor

# Initialize the model and train it
kn = KNeighborsRegressor(n_neighbors=10)
kn.fit(X_train, y_train)

# Predict on val set
y_predict = kn.predict(X_val)

# RMSE
kn_rmse = np.sqrt(mean_squared_error(y_val, y_predict))
kn_rmse

53.17993235102028

After trying all of these models, I think I will stick with RandomForest. Let's now use GridSearchCV to find the optimal hyperparameters. (Took 50 minutes to run, it has been commented out.)

In [208]:
'''
# GridSearch lets us check for the best hyperparameters with a given parameter grid
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Let's GridSearch RandomForest with this param grid
grid_search = GridSearchCV(estimator=rf, 
                           param_grid=param_grid, 
                           cv=3, 
                           n_jobs=-1, 
                           verbose=2, 
                           scoring='neg_mean_squared_error') # MSE is positive or zero, neg is prefix to all mse

# We will attempt to fit all different parameters on the training data.
grid_search.fit(X_train, y_train)

# Best params
best_params = grid_search.best_params_

# Best estimator
best_rf = grid_search.best_estimator_

# Predict on val data
rf_predict = best_rf.predict(X_val)

# RMSE
rf_rmse = np.sqrt(mean_squared_error(y_val, rf_predict))

# Print results
print(rf_rmse)
print(best_rf)
'''

Fitting 3 folds for each of 144 candidates, totalling 432 fits
22.65241381006245
RandomForestRegressor(max_depth=20, n_estimators=200, random_state=1)


This was more done to show I know how GridSearch works. There isn't many hyperparameters with LinearRegression unlike most tree-based models.